In [1]:
import pandas as pd
from pyspark.sql import SparkSession

In [2]:
pd.set_option('display.width', 78)
pd.set_option('display.max_columns',6)

In [3]:
# initiate a Spark session and import CSV data
spark = SparkSession.builder \
   .getOrCreate()

In [4]:
landtemps = spark.read.option("header",True) \
     .csv("data/landtemps.tar.gz")


In [5]:
type(landtemps)

pyspark.sql.dataframe.DataFrame